In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

import gymnasium
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
from cycler import cycler
from gymnasium.utils import seeding
from hydra import compose, initialize
from mpl_toolkits.mplot3d import Axes3D
from omegaconf import DictConfig
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm

import __init__
from scripts.train_rl import setup_environments
from src.data import load
from src.data.loading import ConstantRandomSampler
from src.environments.utils import antialias
from src.evaluation.utils import mm2in
from src.metrics.transforms import AffineTransform, ScaleTranslateTransform
from src.models.sae import assemble_sae
from src.utils import Bunch, deflate, get_display, gl, inflate, print_cfg

sys.modules['gym'] = gymnasium  # see [PR](https://github.com/DLR-RM/stable-baselines3/pull/780)
from stable_baselines3 import SAC

---

#### Plotting setup

In [ ]:
plt.rcParams.update({
    'axes.prop_cycle': cycler('color', ["#0173B2", "#DE8F05", "#029E73", "#D55E00", "#CC78BC",
                                        "#CA9161", "#FBAFE4", "#949494", "#ECE133", "#56B4E9"]),
    'axes.titlepad': 3.0,
    'axes.xmargin': 0.025,
    'axes.ymargin': 0.025,
    'axes.titlesize': 'medium',
    'axes.labelpad': 1.0,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'font.family': 'serif',
    'font.size': 8,
    'text.usetex': True,
    'text.latex.preamble': [r'\usepackage{lmodern}'],
    'grid.alpha': 0.1,
    'grid.color': '#000000',
    'legend.borderaxespad': 0.25,
    'legend.borderpad': 0.0,
    'legend.frameon': False,
    'legend.columnspacing': 1.0,
    'legend.handletextpad': 0.5,
    'legend.handlelength': 0.75,
    'lines.solid_capstyle': 'round',
    'lines.solid_joinstyle': 'round',
    'xtick.major.pad': 2.0,
    'xtick.major.size': 2.0,
    'xtick.minor.size': 1.0,
    'ytick.major.pad': 2.0,
    'ytick.major.size': 2.0,
    'ytick.minor.size': 1.0,
    'figure.constrained_layout.h_pad': 0.0,
    'figure.constrained_layout.hspace': 0.0,
    'figure.constrained_layout.use': True,
    'figure.constrained_layout.w_pad': 0.0,
    'figure.constrained_layout.wspace': 0.0
})

---

In [ ]:
with initialize(version_base=None, config_path='../configs'):
    rl_cfg = compose(config_name='train_rl', overrides=[
        '+experiment=rl-feat',
        'training.observation.keypoints=True',
        'training.sae_checkpoint=logs/sae/panda_push_custom/keynet+keynet/2023-03-16--02-11-09--34050912/checkpoint_final.pth',
        'training.sae_name=autumn-resonance-526',
        'training.sae_experiment=sae-keynet-vel-var-bg',
        'wandb=off',
        'hydra=hush'
    ])

with initialize(version_base=None, config_path='../configs'):
    sae_cfg = compose(config_name='train_sae', overrides=[
        '+experiment=sae-keynet-vel-var-bg',
        'wandb=off',
        'hydra=hush'
    ])

---

In [ ]:
checkpoint = Bunch(**torch.load('../logs/sae/panda_push_custom/keynet+keynet/2023-03-16--02-11-09--34050912/checkpoint_final.pth', map_location=gl.device))

# reinstantiate model and optimizer
model = assemble_sae(sae_cfg)
model.load_state_dict(checkpoint.model_state_dict)

In [ ]:
# load datasets
dataset_valid, = load(sae_cfg, valid=True)

loader_valid = DataLoader(dataset_valid, sae_cfg.training.batch_size,
                          sampler=ConstantRandomSampler(dataset_valid, sae_cfg.dataset.seed),
                          shuffle=False, drop_last=True, num_workers=8, pin_memory=False)

In [ ]:
with torch.no_grad():
    model.eval()  # put model into evaluation state
    track_fps = []  # feature points for tracking error computation
    track_kps = []  # site coordinates for tracking error computation

    # loop over validation batches
    for batch, (inputs, _, sites) in enumerate(tqdm(loader_valid, leave=False)):

        # move all data to GPU
        inputs = inputs.to(gl.device)

        # encoder pass to obtain feature points
        fps = model.encoder(deflate(inputs))
        feature_points = inflate(fps, len(inputs))

        # storing fps and sites for first image of each snippet (avoiding duplicates)
        track_fps.append(feature_points[:, 0])
        track_kps.append(sites[:, 0])

    track_fps = torch.cat(track_fps).to('cpu')
    track_kps = torch.cat(track_kps).to('cpu')

In [ ]:
n_sites = track_kps.shape[1]
n_fps = track_fps.shape[1]

pairwise_errors = torch.ones((n_sites, n_fps)) * np.inf
regrs = [None] * n_sites
closest_fps = [None] * n_sites

# compute error for each pair of site and feature point
for site in range(n_sites):
    for fp in range(n_fps):
        regr = AffineTransform()
        regr.fit(track_fps[:, fp], track_kps[:, site])  # fit transformation
        error = regr.mse(track_fps[:, fp], track_kps[:, site])
        if torch.all(pairwise_errors[site, :] >= error):
            regrs[site] = regr
            closest_fps[site] = fp
        pairwise_errors[site, fp] = error

---

In [ ]:
policy = SAC.load('../logs/rl/PandaPush-custom/2023-03-18--19-49-40--34100344/final_model.zip')

In [ ]:
venv = setup_environments(rl_cfg, 1)

In [ ]:
def run_episode(seed):
    _ = venv.reset()
    _ = venv.seed(seed)
    action = np.array([[0, 0, 0]])
    dones = np.array([False])

    image = venv.render()
    observations = []

    while not np.any(dones):
        obs, _, dones, info = venv.step(action)
        if np.any(dones):
            observations.append(info[0]['terminal_observation'])
        else:
            observations.append(obs.copy())
        obs.pop('keypoints')
        action, _ = policy.predict(obs, deterministic=True)

    return image, observations

In [ ]:
def plot_trajectories(ax, image, observations, keypoints):

    ax.imshow(antialias(torch.tensor(image / 255.0), 2), interpolation='none')
    kps = (np.stack([np.squeeze(observations[i]['keypoints']) for i in range(len(observations))], axis=1) + 1) * 128
    fps_orig = np.stack([np.squeeze(observations[i]['feature_points']) for i in range(len(observations))], axis=1)
    fps = (fps_orig + 1) * 128

    for kp_idx in keypoints:
        ax.plot(kps[kp_idx, :, 0], kps[kp_idx, :, 1], color='w', marker='.', lw=1, markersize=2)
    for fp_idx in [closest_fps[k] for k in keypoints]:
        ax.plot(fps[fp_idx, :, 1], fps[fp_idx, :, 0], color='C3', marker='.', lw=1, markersize=2)
        
    selected_regrs = [regrs[k] for k in keypoints]
    for i, fp_idx in enumerate([closest_fps[k] for k in keypoints]):
        fps_t = (selected_regrs[i].transform(torch.tensor(fps_orig[fp_idx])) + 1) * 128
        ax.plot(fps_t[:, 1], fps_t[:, 0], color='C0', marker='.', lw=1, markersize=2)

    ax.set_xlim(31.5, 223.5)
    ax.set_ylim(191.5, -0.5)

    return ax

In [ ]:
fig1, axes1 = plt.subplots(1, 4)
fig2, axes2 = plt.subplots(1, 4)

for i, seed in enumerate([13, 3, 4, 14]):
    image, observations = run_episode(seed)  # 5, 7, 11, 13, 28

    axes1[i] = plot_trajectories(axes1[i], image, observations, [4])
    axes1[i].axis('off')

    axes2[i] = plot_trajectories(axes2[i], image, observations, [0])
    axes2[i].axis('off')

fig1.set_size_inches(mm2in(122, 28.67))
fig1.savefig('../local/paper/img_trajectories_endeffector.pdf')

fig2.set_size_inches(mm2in(122, 28.67))
fig2.savefig('../local/paper/img_trajectories_object.pdf')

---

In [ ]:
_ = venv.reset()
_ = venv.seed(13)
action = np.array([[0, 0, 0]])
dones = np.array([False])

images = []
observations = []

while not np.any(dones):
    images.append(venv.render())
    obs, _, dones, info = venv.step(action)
    if np.any(dones):
        observations.append(info[0]['terminal_observation'])
    else:
        observations.append(obs.copy())
    obs.pop('keypoints')
    action, _ = policy.predict(obs, deterministic=True)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(antialias(torch.tensor(images[0] / 255.0), 2), interpolation='none')

kps = (np.squeeze(observations[0]['keypoints']) + 1) * 128
ax.scatter(kps[[0, 1, 4], 0], kps[[0, 1, 4], 1], color='w', marker='.', s=10)

ax.set_xlim(31.5, 223.5)
ax.set_ylim(191.5, -0.5)
ax.axis('off')

fig.set_size_inches(mm2in(122 * 0.15, 122 * 0.15))
fig.savefig('../local/paper/img_pandapush_start.pdf')

In [ ]:
fig, ax = plt.subplots()
ax.imshow(antialias(torch.tensor(images[6] / 255.0), 2), interpolation='none')

# kps = (np.squeeze(observations[5]['keypoints']) + 1) * 128
# ax.scatter(kps[[0, 1, 4], 0], kps[[0, 1, 4], 1], color='w', marker='.', lw=1)

ax.set_xlim(31.5, 223.5)
ax.set_ylim(191.5, -0.5)
ax.axis('off')

fig.set_size_inches(mm2in(122 * 0.15, 122 * 0.15))
fig.savefig('../local/paper/img_pandapush_mid.pdf')

In [ ]:
fig, ax = plt.subplots()
ax.imshow(antialias(torch.tensor(images[-1] / 255.0), 2), interpolation='none')

# kps = (np.squeeze(observations[-2]['keypoints']) + 1) * 128
# ax.scatter(kps[[0, 1, 4], 0], kps[[0, 1, 4], 1], color='w', marker='.', lw=1)

ax.set_xlim(31.5, 223.5)
ax.set_ylim(191.5, -0.5)
ax.axis('off')

fig.set_size_inches(mm2in(122 * 0.15, 122 * 0.15))
fig.savefig('../local/paper/img_pandapush_end.pdf')

---
---

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.voxels(np.ones([1, 1, 1]), alpha=0.5)
ax.scatter3D(*(0.5 * np.ones(3)), color='k', s=50)
ax.scatter3D(*np.ones(3), color='C3', s=50)
ax.quiver(*(0.5 * np.ones(3)), *(0.5 * np.ones(3)), capstyle='round')
ax.axis('off')

fig.set_size_inches(mm2in(122 * 0.25, 30.2))
# fig.savefig(f'../local/paper/img_offset3d.pdf')